<a href="https://colab.research.google.com/github/hamletbatista/sej/blob/master/Predicting_Successful_Content_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Predicting Successful Content with Python.

* Let's find the 15% of search queries that Google has never seen before
* Group them into canonical queries (consolidate duplicates)
* manually group intents
* Finally forecast their traffic intents 30, 60, 90 days to pick the most promising candidates



In [ ]:
%%capture
!pip install git+https://github.com/joshcarty/google-searchconsole

First, there is some setup to download a client_id.json file our Python code can use to connect securely to Google Search Console.

Activate Search Console API in Compute Engine https://console.cloud.google.com/apis/api/webmasters.googleapis.com/overview?project=&folder=&organizationId= Create New Credentials / Help me choose (Search Console API, Other UI, User data) https://console.cloud.google.com/apis/credentials/wizard?api=iamcredentials.googleapis.com&project= Download client_id.json

In [ ]:
#upload client_id.json and credentials.json files
from google.colab import files

names = files.upload()

In [ ]:
#names

In [ ]:
filename=list(names.keys())[0]

In [ ]:
filename

In [ ]:
import searchconsole

account = searchconsole.authenticate(client_config=filename, serialize='credentials.json', flow="console")

In [ ]:
account.webproperties

In [ ]:
domain_name = "https://www.sitename.com/" #@param {type:"string"}


In [ ]:
#Insert your domain name below.
webproperty = account[domain_name]

Line below should print the the site's property

In [ ]:
webproperty

In [ ]:
#let's build a pandas dataframe with the search console data
import pandas as pd

def get_search_console_data(webproperty, days=-365):
  if webproperty is not None:
    query = webproperty.query.range(start='today', days=days).dimension('date', 'query')
    r = query.get()
    df = pd.DataFrame(r.rows)
    return df

  print("Web property doesn't exist, please select a valid one from this list")
  print(account.webproperties)

  return None

In [ ]:
df = get_search_console_data(webproperty)

In [ ]:
df.head()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1810000 entries, 0 to 1809999
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   date         object 
 1   query        object 
 2   clicks       int64  
 3   impressions  int64  
 4   ctr          float64
 5   position     float64
dtypes: float64(2), int64(2), object(2)
memory usage: 82.9+ MB


In [ ]:
df.to_csv("canadahelps.csv")

In [ ]:
!gzip canadahelps.csv

In [ ]:
df["date"] = pd.to_datetime(df.date)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1810000 entries, 0 to 1809999
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   date         datetime64[ns]
 1   query        object        
 2   clicks       int64         
 3   impressions  int64         
 4   ctr          float64       
 5   position     float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(1)
memory usage: 82.9+ MB


Most recent data is 2 days old

In [ ]:
df[df["date"] > "2020-11-6"]

In [ ]:
last_day_queries = df[df["date"] > "2020-11-6"]["query"]

In [ ]:
len(last_day_queries)

5000

In [ ]:
rest_of_queries = df[df["date"] < "2020-11-6"]["query"]

In [ ]:
len(rest_of_queries)

1800000

Next, we want to find the queries in last day, but not in the rest.

In [ ]:
fiften_percent = set(last_day_queries) - set(rest_of_queries)

In [ ]:
len(fiften_percent)

212

In [ ]:
fiften_percent

Let's check if these queries are semantic duplicates of existing ones.


In [ ]:
%%capture
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')



100%|██████████| 245M/245M [02:27<00:00, 1.66MB/s]


In [ ]:
# Two lists of sentences
sentences1 = ['The cat sits outside',
             'A man is playing guitar',
             'The new movie is awesome']

sentences2 = ['The dog plays in the garden',
              'A woman watches TV',
              'The new movie is so great']



In [ ]:
#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)



In [ ]:
#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

The cat sits outside 		 The dog plays in the garden 		 Score: 0.2166
A man is playing guitar 		 A woman watches TV 		 Score: -0.0146
The new movie is awesome 		 The new movie is so great 		 Score: 0.9816


Next, let's try with our queries

In [ ]:
fiften_percent_list = list(fiften_percent)

In [ ]:

#Compute embedding for both lists
embeddings1 = model.encode(fiften_percent_list, convert_to_tensor=True)



In [ ]:
# try on a smaller set, as it takes too long to run on full set of +1m queries
rest_of_queries_list = list(set(rest_of_queries))[:10000]

In [ ]:
embeddings2 = model.encode( rest_of_queries_list, convert_to_tensor=True)


In [ ]:
#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)


In [ ]:
#Output the pairs with their score
for i in range(len(fiften_percent_list)):
    score = cosine_scores[i][i]

    if score > 0.4:
        print(f"{i}. {fiften_percent_list[i]} <> {rest_of_queries_list[i]} \nScore: {score:.4f}")


Once we have the duplicate queries, we can use their historical traffic to predict the potential traffic of the new ones and prioritize the topics to focus on.

Loading from backup

In [ ]:
df.head()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1810000 entries, 0 to 1809999
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   date         object 
 1   query        object 
 2   clicks       int64  
 3   impressions  int64  
 4   ctr          float64
 5   position     float64
dtypes: float64(2), int64(2), object(2)
memory usage: 96.7+ MB


In [ ]:
ideas_df = df[df['query'].str.contains("idea")]

In [ ]:
ideas_df["date"] = pd.to_datetime(ideas_df["date"])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
ideas_df

In [ ]:
near_me_df = df[df['query'].str.contains("near me")]

In [ ]:
near_me_df["date"] = pd.to_datetime(near_me_df["date"])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
ideas_df = ideas_df.set_index("date")

In [ ]:
near_me_df = near_me_df.set_index("date")

In [ ]:
ideas_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 16823 entries, 2020-04-16 to 2020-11-07
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   query        16823 non-null  object 
 1   clicks       16823 non-null  int64  
 2   impressions  16823 non-null  int64  
 3   ctr          16823 non-null  float64
 4   position     16823 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 788.6+ KB


In [ ]:
near_me_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3806 entries, 2019-12-23 to 2020-11-07
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   query        3806 non-null   object 
 1   clicks       3806 non-null   int64  
 2   impressions  3806 non-null   int64  
 3   ctr          3806 non-null   float64
 4   position     3806 non-null   float64
dtypes: float64(2), int64(2), object(1)
memory usage: 178.4+ KB


In [ ]:
ideas_df.head()

,query,clicks,impressions,ctr,position
date,,,,,
2020-04-16,virtual fundraising ideas,74,306,0.241830,4.336601
2020-04-21,virtual fundraising ideas,64,334,0.191617,4.667665
2020-04-14,virtual fundraising ideas,56,359,0.155989,5.810585
2020-04-15,virtual fundraising ideas,55,311,0.176849,6.514469
2020-04-23,virtual fundraising ideas,50,320,0.156250,5.618750


In [ ]:
near_me_df.head()

,query,clicks,impressions,ctr,position
date,,,,,
2019-12-23,orphanage near me,3,5,0.600000,1.000000
2020-01-29,charities near me,3,16,0.187500,6.125000
2020-05-31,charities near me,3,10,0.300000,4.700000
2020-10-01,charities near me,3,11,0.272727,3.454545
2020-10-21,charities near me,3,14,0.214286,4.571429


In [ ]:
grouped_ideas_df = ideas_df.groupby(pd.Grouper(freq='M')).sum()[["clicks", "impressions"]]

In [ ]:
grouped_near_me_df = near_me_df.groupby(pd.Grouper(freq='M')).sum()[["clicks", "impressions"]]

In [ ]:
grouped_ideas_df

,clicks,impressions
date,,
2019-11-30,463,9551
2019-12-31,150,10425
2020-01-31,111,13184
2020-02-29,85,11717
2020-03-31,165,12928
2020-04-30,1824,26301
2020-05-31,1600,27000
2020-06-30,961,22298
2020-07-31,655,16746


In [ ]:
grouped_near_me_df

,clicks,impressions
date,,
2019-11-30,58,1324
2019-12-31,119,2368
2020-01-31,108,1844
2020-02-29,116,1876
2020-03-31,90,2496
2020-04-30,134,20113
2020-05-31,139,2347
2020-06-30,105,3016
2020-07-31,92,3414


In [ ]:
import plotly.express as px


In [ ]:
fig = px.line(grouped_ideas_df, y="clicks", title='Clicks over Time for Ideas')


In [ ]:
fig.show()

In [ ]:
fig = px.line(grouped_near_me_df, y="clicks", title='Clicks over Time for Near Me')

In [ ]:
fig.show()

In [ ]:
grouped_ideas_df = ideas_df.groupby(pd.Grouper(freq='D')).sum()[["clicks", "impressions"]]

In [ ]:
grouped_ideas_df

,clicks,impressions
date,,
2019-11-12,38,560
2019-11-13,38,601
2019-11-14,27,563
2019-11-15,24,482
2019-11-16,16,380
...,...,...
2020-11-03,54,1244
2020-11-04,74,1248
2020-11-05,52,1130


In [ ]:
fig = px.line(grouped_ideas_df, y="clicks", title='Clicks over Time for Ideas')


In [ ]:
fig.show()

In [ ]:
grouped_near_me_df = near_me_df.groupby(pd.Grouper(freq='D')).sum()[["clicks", "impressions"]]

In [ ]:
grouped_near_me_df

,clicks,impressions
date,,
2019-11-12,2,104
2019-11-13,2,46
2019-11-14,2,51
2019-11-15,1,59
2019-11-16,2,98
...,...,...
2020-11-03,8,162
2020-11-04,8,126
2020-11-05,10,261


In [ ]:
fig = px.line(grouped_near_me_df, y="clicks", title='Clicks over Time for Near Me')


In [ ]:
fig.show()

In [ ]:
from fbprophet import Prophet
from fbprophet.plot import plot_plotly

In [ ]:
grouped_ideas_df.reset_index()

,date,clicks,impressions
0,2019-11-12,38,560
1,2019-11-13,38,601
2,2019-11-14,27,563
3,2019-11-15,24,482
4,2019-11-16,16,380
...,...,...,...
357,2020-11-03,54,1244
358,2020-11-04,74,1248
359,2020-11-05,52,1130
360,2020-11-06,49,1006


Rename columns

In [ ]:
dft = grouped_ideas_df.reset_index().rename(columns={"date":"ds", "clicks":"y"})

In [ ]:
dft

,ds,y,impressions
0,2019-11-12,38,560
1,2019-11-13,38,601
2,2019-11-14,27,563
3,2019-11-15,24,482
4,2019-11-16,16,380
...,...,...,...
357,2020-11-03,54,1244
358,2020-11-04,74,1248
359,2020-11-05,52,1130
360,2020-11-06,49,1006


In [ ]:
m = Prophet()
m.fit(dft)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
#Predicting clicks for the next 30 days.
future_30 = m.make_future_dataframe(periods=30)
forecast_30 = m.predict(future_30)
#Predicting clicks for the next 60 days.
future_60 = m.make_future_dataframe(periods=60)
forecast_60 = m.predict(future_60)
#Predicting clicks for the next 90 days.
future_90 = m.make_future_dataframe(periods=90)
forecast_90 = m.predict(future_90)

In [ ]:
#Visualizing the prediction for next 30 days.
plot_plotly(m, forecast_30, xlabel='Date', ylabel='Clicks')

In [ ]:
#Visualizing the prediction for next 60 days.
plot_plotly(m, forecast_60, xlabel='Date', ylabel='Clicks')

In [ ]:
#Visualizing the prediction for next 90 days.
plot_plotly(m, forecast_90, xlabel='Date', ylabel='Clicks')